In [ ]:
import marimo as mo

In [ ]:
import os
from typing import Self
from pathlib import Path
import pydantic
import highspy
from ortools.sat.python import cp_model

# 資源制約付きプロジェクトスケジューリング問題

### 入力データ

- ジョブの集合 $\mathrm{Job}$, 添字は $j, k$
- 資源の集合 $\mathrm{Res}$, 添字は $r$
- ジョブ間の時間成約を表す集合 $\mathrm{Prec} \subset \mathrm{Job} \times \mathrm{Job}$
    - $(j,k) \in \mathrm{Prec}$ のとき ジョブ $j$ とジョブ $k$ の時刻間に何かしらの関係がある.
- 最大の期数 $T$, 添字は $t, s \in \{1, \dots, T \}$
    - 期間 $t$ は時刻 $t-1$ から時刻 $t$ までであるとする.
- ジョブ $j$ の処理時間 $p_j$
- ジョブ $j$ を期 $t$ に開始したときの費用 $\mathrm{Cost}_{jt}$
- ジョブ $j$ の開始後 $t$ 期経過時の処理に要する資源 $r$ の量 $a_{jrt}$
- 期 $t$ における資源 $r$ の使用可能量上限 $\mathrm{RUB}_{rt}$

### 変数

- $x_{jt} \in \{ 0, 1 \}$: ジョブ $j$ を期 $t$ に開始するとき $1$, それ以外は $0$

### 目的関数

$$
\min \sum_{j \in \mathrm{Job}} \sum_{t=1}^{T-p_j+1} \mathrm{Cost}_{jt} x_{jt}
$$

### 制約条件

- ジョブ遂行成約:
  $\sum_{t=1}^{T-p_j+1} x_{jt} = 1 \quad (\forall j \in \mathrm{Job})$
- 資源成約:
  $\sum_{j \in \mathrm{Job}} \sum_{s = \max(t - p_j + 1, 1)}^{\min(t, T - p_j + 1)} a_{jr,t-s} x_{js} \leq \mathrm{RUB}_{rt} \quad (\forall r \in \mathrm{Res}, \forall t \in \{ 1, \dots, T \})$
- 時間制約: $\sum_{t=2}^{T-p_j+1} (t-1)x_{jt} + p_j \leq \sum_{t=2}^{T-p_k+1} (t-1) x_{kt} \quad (\forall (j,k) \in \mathrm{Prec})$

In [ ]:
# https://scmopt.github.io/manual/15mypulp.html#multidict%E9%96%A2%E6%95%B0

def multidict(d: dict):
    ret = [list(d.keys())]
    for k, arr in d.items():
        if type(arr) is not list:
            arr = [arr]
        append_num = 1 + len(arr) - len(ret)
        if append_num > 0:
            ret = ret + [{} for _ in range(append_num)]
        for i, val in enumerate(arr):
            ret[i + 1][k] = val
    return ret

In [ ]:
def make_1r():
    J, p = multidict(
        {  # jobs, processing times
            1: 1,
            2: 3,
            3: 2,
            4: 2,
        }
    )
    P = [(1, 2), (1, 3), (2, 4)]
    R = [1]
    T = 6
    c = {}
    for j in J:
        for t in range(1, T - p[j] + 2):
            c[j, t] = 1 * (t - 1 + p[j])
    a = {
        (1, 1, 0): 2,
        (2, 1, 0): 2,
        (2, 1, 1): 1,
        (2, 1, 2): 1,
        (3, 1, 0): 1,
        (3, 1, 1): 1,
        (4, 1, 0): 1,
        (4, 1, 1): 2,
    }
    RUB = {(1, 1): 2, (1, 2): 2, (1, 3): 1, (1, 4): 2, (1, 5): 2, (1, 6): 2}
    return (J, P, R, T, p, c, a, RUB)

In [ ]:
def make_2r():
    J, p = multidict(
        {  # jobs, processing times
            1: 2,
            2: 2,
            3: 3,
            4: 2,
            5: 5,
        }
    )
    P = [(1, 2), (1, 3), (2, 4)]
    R = [1, 2]
    T = 6
    c = {}
    for j in J:
        for t in range(1, T - p[j] + 2):
            c[j, t] = 1 * (t - 1 + p[j])
    a = {
        # resource 1:
        (1, 1, 0): 2,
        (1, 1, 1): 2,
        (2, 1, 0): 1,
        (2, 1, 1): 1,
        (3, 1, 0): 1,
        (3, 1, 1): 1,
        (3, 1, 2): 1,
        (4, 1, 0): 1,
        (4, 1, 1): 1,
        (5, 1, 0): 0,
        (5, 1, 1): 0,
        (5, 1, 2): 1,
        (5, 1, 3): 0,
        (5, 1, 4): 0,
        # resource 2:
        (1, 2, 0): 1,
        (1, 2, 1): 0,
        (2, 2, 0): 1,
        (2, 2, 1): 1,
        (3, 2, 0): 0,
        (3, 2, 1): 0,
        (3, 2, 2): 0,
        (4, 2, 0): 1,
        (4, 2, 1): 2,
        (5, 2, 0): 1,
        (5, 2, 1): 2,
        (5, 2, 2): 1,
        (5, 2, 3): 1,
        (5, 2, 4): 1,
    }
    RUB = {
        (1, 1): 2,
        (1, 2): 2,
        (1, 3): 2,
        (1, 4): 2,
        (1, 5): 2,
        (1, 6): 2,
        (1, 7): 2,
        (2, 1): 2,
        (2, 2): 2,
        (2, 3): 2,
        (2, 4): 2,
        (2, 5): 2,
        (2, 6): 2,
        (2, 7): 2,
    }
    return (J, P, R, T, p, c, a, RUB)

## HiGHS によるモデリング

In [ ]:
class Model1Highs:
    def __init__(self, J, P, R, T, p, c, a, RUB):
        self.model = highspy.Highs()

        # s - start time variable
        # x=1 if job j starts on period t
        self.s, self.x = {}, {}
        for j in J:
            self.s[j] = self.model.addVariable()
            for t in range(1, T - p[j] + 2):
                self.x[j, t] = self.model.addBinary()

        for j in J:
            # job execution constraints
            self.model.addConstr(
                sum(self.x[j, t] for t in range(1, T - p[j] + 2)) == 1
            )

            # start time constraints
            self.model.addConstr(
                sum((t - 1) * self.x[j, t] for t in range(2, T - p[j] + 2)) == self.s[j]
            )

        # resource upper bound constraints
        for t in range(1, T+1):
            for r in R:
                self.model.addConstr(
                    sum(
                        a[j, r, t - t_] * self.x[j, t_]
                        for j in J
                        for t_ in range(max(t - p[j] + 1, 1), min(t + 1, T - p[j] + 2))
                    )
                    <= RUB[r, t]
                )

        # time (precedence) constraints, i.e., s[k]-s[j] >= p[j]
        for (j, k) in P:
            self.model.addConstr(self.s[k] - self.s[j] >= p[j])

        self.objective = sum(c[j, t] * self.x[j, t] for (j, t) in self.x)
        # self.model.minimize(self.objective)

    def solve(self) -> None:
        # self.model.run()
        self.model.minimize(self.objective)
        self.solution = self.model.getSolution()
        self.info = self.model.getInfo()

In [ ]:
(J1, P1, R1, T1, p1, c1, a1, RUB1) = make_1r()
model1 = Model1Highs(J1, P1, R1, T1, p1, c1, a1, RUB1)

model1.solve()

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 17 rows; 24 cols; 84 nonzeros; 20 integer variables (20 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+00]
  Cost   [1e+00, 6e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 3e+00]
Presolving model
17 rows, 21 cols, 72 nonzeros  0s
14 rows, 18 cols, 67 nonzeros  0s
12 rows, 14 cols, 59 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol          

In [ ]:
print (f"Opt.value = {model1.info.objective_function_value}")

for (_j, _t) in model1.x:
    _val = model1.solution.col_value[model1.x[_j, _t].index]
    if _val > 0.5:
        print(f"x[{_j},{_t}] = {_val}")

for _j in model1.s:
    _val = model1.solution.col_value[model1.s[_j].index]
    print(f"s[{_j}] = {_val}")

Opt.value = 16.0
x[1,1] = 1.0
x[2,2] = 1.0
x[3,4] = 1.0
x[4,5] = 1.0
s[1] = 0.0
s[2] = 1.0
s[3] = 3.0
s[4] = 4.0


In [ ]:
(J2, P2, R2, T2, p2, c2, a2, RUB2) = make_2r()
model2 = Model1Highs(J2, P2, R2, T2, p2, c2, a2, RUB2)

model2.solve()

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 25 rows; 26 cols; 127 nonzeros; 21 integer variables (21 binary)
Coefficient ranges:
  Matrix [1e+00, 4e+00]
  Cost   [2e+00, 6e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 2e+00]
Presolving model
23 rows, 20 cols, 94 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0      

In [ ]:
print (f"Opt.value = {model2.info.objective_function_value}")

for (_j, _t) in model2.x:
    _val = model2.solution.col_value[model2.x[_j, _t].index]
    if _val > 0.5:
        print(f"x[{_j},{_t}] = {_val}")

for _j in model2.s:
    _val = model2.solution.col_value[model2.s[_j].index]
    print(f"s[{_j}] = {_val}")

Opt.value = 23.0
x[1,1] = 1.0
x[2,3] = 1.0
x[3,4] = 1.0
x[4,5] = 1.0
x[5,1] = 1.0
s[1] = 0.0
s[2] = 2.0
s[3] = 3.0
s[4] = 4.0
s[5] = 0.0


## インスタンス

### kobe-scheduling

- リンク: https://github.com/ptal/kobe-scheduling
- データフォーマットはデータによって異なり, 特に説明もない.
  https://github.com/ptal/kobe-scheduling/data/rcpsp/patterson.rcp であれば (多分)
    - 1 行目: ジョブの数, リソースの種類
    - 3 行目: 各リソースの上限値
    - 5 行目以降: ジョブの情報が並ぶ.
      処理時間, [リソースの消費数, ...], 後続ジョブ数, [後続ジョブ番号, ...]
- 目的関数は makespan (多分)

In [ ]:
parent = str(Path(os.path.abspath(__file__)).parent)
data_dir = Path(parent, "kobe-scheduling", "data", "rcpsp", "patterson.rcp")

In [ ]:
class Job(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(frozen=True)
    id: int
    time: int
    res_usages: list[int]

class Resource(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(frozen=True)
    ub: int

class Condition(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(frozen=True)
    jobs: list[Job]
    ress: list[Resource]
    prec: set[tuple[int,int]]

    @staticmethod
    def from_file(filepath) -> Self:
        prec = set()
        with open(filepath) as f:
            _njobs, _nress = map(lambda s: int(s), f.readline().split())

            f.readline()

            resubs = list(map(lambda s: int(s), f.readline().split()))

            ress = [Resource(ub=resub) for resub in resubs]

            jobs = []
            job_id = 0
            while (line := f.readline()):
                datas = list(map(lambda s: int(s), line.split()))
                if len(datas) == 0:
                    continue

                idx = 0
                time = datas[idx]
                idx += 1
                res_usages = [datas[idx + jdx] for jdx in range(_nress)]
                idx += _nress

                # 次から始まる数値列の長さなのでスキップ
                idx += 1

                while idx < len(datas):
                    prec.add((job_id, datas[idx] - 1))
                    idx += 1

                jobs.append(Job(id=job_id, time=time, res_usages=res_usages))
                job_id += 1

        return Condition(jobs=jobs, ress=ress, prec=prec)

In [ ]:
_filepath = data_dir / Path("pat1.rcp")
cond1 = Condition.from_file(_filepath)

In [ ]:
cond1.model_dump()

In [ ]:
class Model2Highs:
    def __init__(self, cond: Condition):
        self.model = highspy.Highs()

        horizon = sum(job.time for job in cond.jobs)

        # s[j]: ジョブ j の開始期
        self.s = [self.model.addVariable(ub=horizon - 1) for job in cond.jobs]
        # e[j]: ジョブ j の終了期 "の次の期"
        self.e = [self.s[jdx] + job.time for jdx, job in enumerate(cond.jobs)]
        # x[j][t]: ジョブ j を期 t に開始する時だけ 1
        self.x = [[self.model.addBinary() for t in range(horizon)] for job in cond.jobs]

        # 各ジョブは必ず処理される
        for jdx, job in enumerate(cond.jobs):
            self.model.addConstr(
                sum(self.x[jdx][t] for t in range(horizon - max(0, job.time - 1))) == 1
            )
            for t in range(horizon - max(0, job.time - 1), horizon):
                self.model.addConstr(self.x[jdx][t] == 0)

        # s と x の関係
        self.model.addConstrs(
            [
                sum(t * self.x[jdx][t] for t in range(horizon)) == self.s[jdx]
                for jdx, _ in enumerate(cond.jobs)
            ]
        )

        # ジョブ依存関係
        for idx, jdx in cond.prec:
            self.model.addConstr(self.e[idx] <= self.s[jdx])

        # 資源制約
        for id_res, res in enumerate(cond.ress):
            for t in range(horizon):
                self.model.addConstr(
                    sum(
                        sum(
                            self.x[id_job][_t]
                            for _t in range(max(0, t - job.time + 1), t + 1)
                        ) * job.res_usages[id_res]
                        for id_job, job in enumerate(cond.jobs)
                    ) <= res.ub
                )

        # 目的関数: makespan
        self.objective = self.model.addVariable(ub=horizon)
        for jdx, job in enumerate(cond.jobs):
            self.model.addConstr(self.objective >= self.e[jdx])

    def solve(self) -> None:
        # self.model.run()
        self.model.minimize(self.objective)
        self.solution = self.model.getSolution()
        self.info = self.model.getInfo()

In [ ]:
model3 = Model2Highs(cond1)
model3.solve()

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 210 rows; 575 cols; 2563 nonzeros; 560 integer variables (560 binary)
Coefficient ranges:
  Matrix [1e+00, 4e+01]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 4e+01]
  RHS    [1e+00, 6e+00]
Presolving model
182 rows, 547 cols, 2442 nonzeros  0s


154 rows, 350 cols, 1877 nonzeros  0s
145 rows, 350 cols, 1894 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   145 rows
   350 cols (335 binary, 0 integer, 15 implied int., 0 continuous, 0 domain fixed)
   1894 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   18              inf                  inf        0      0      0         0     0.0s
         0       0         0   0.00%   18              inf                  inf        0      0      5        64     0.0s


 R       0       0         0 100.00%   19              19                 0.00%      449     25     18       145     0.1s
         1       0         1 100.00%   19              19                 0.00%      336     25     18       145     0.1s

Solving report
  Status            Optimal
  Primal bound      19
  Dual bound        19
  Gap               0% (tolerance: 0.01%)
  P-D integral      0
  Solution status   feasible
                    19 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.06 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 0
  Nodes             1
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     145 (total)
                    0 (strong br.)
                    81 (separation)
                    0 (heuristics)


In [ ]:
print (f"Opt.value = {model3.info.objective_function_value}")

for _id_job, s in enumerate(model3.s):
    _val = model3.solution.col_value[s.index]
    print(f"s[{_id_job}] = {_val}")

Opt.value = 19.0
s[0] = 0.0
s[1] = 0.0
s[2] = 0.0
s[3] = 1.0
s[4] = 5.0
s[5] = 5.0
s[6] = 6.0
s[7] = 13.0
s[8] = 14.0
s[9] = 6.0
s[10] = 9.0
s[11] = 11.0
s[12] = 14.0
s[13] = 19.0


In [ ]:
class Model2CpSat:
    def __init__(self, cond: Condition):
        self.model = cp_model.CpModel()

        horizon = sum(job.time for job in cond.jobs)

        self.starts = [self.model.new_int_var(lb=0, ub=horizon-job.time, name="") for job in cond.jobs]
        self.jobs = [
            self.model.new_fixed_size_interval_var(self.starts[id_job], job.time, name="")
            for id_job, job in enumerate(cond.jobs)
        ]

        # ジョブ間依存関係
        for idx, jdx in cond.prec:
            self.model.add(self.jobs[idx].end_expr() <= self.jobs[jdx].start_expr())

        # 資源制約
        for id_res, res in enumerate(cond.ress):
            capacity = res.ub
            intervals = []
            demands = []
            for id_job, job in enumerate(cond.jobs):
                if job.res_usages[id_res] == 0:
                    continue
                intervals.append(self.jobs[id_job])
                demands.append(job.res_usages[id_res])

            self.model.add_cumulative(intervals, demands, capacity)

        # 目的関数: makespan
        self.objective = self.model.new_int_var(lb=0, ub=horizon, name="")
        self.model.add_max_equality(self.objective, [interval.end_expr() for interval in self.jobs])
        self.model.minimize(self.objective)

    def solve(self, timeout: int = 180):
        self.solver = cp_model.CpSolver()
        self.solver.parameters.log_search_progress = True
        self.solver.parameters.max_time_in_seconds = timeout
        self.status = self.solver.solve(self.model)

In [ ]:
model4 = Model2CpSat(cond1)
model4.solve()


Starting CP-SAT solver v9.13.4784
Parameters: max_time_in_seconds: 180 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x274e8745b400f59d)
#Variables: 15 (#ints: 1 in objective) (14 primary variables)
  - 2 in [0,34]
  - 1 in [0,35]
  - 2 in [0,36]
  - 3 in [0,37]
  - 2 in [0,38]
  - 2 in [0,39]
  - 3 in [0,40]
#kCumulative: 3 (#intervals: 9)
#kInterval: 14
#kLinMax: 1 (#expressions: 14)
#kLinear2: 20

Starting presolve at 0.00s
  1.33e-05s  0.00e+00d  [DetectDominanceRelations] 
  5.17e-04s  0.00e+00d  [PresolveToFixPoint] #num_loops=8 #num_dual_strengthening=4 
  1.29e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  3.23e-06s  0.00e+00d  [DetectDuplicateColumns] 
  1.21e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 63 nodes and 75 arcs.
[Symmetry] Symmetry computation done. time: 1.4217e-05 dtime: 7.23e-06
  8.68e-06s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  8.

In [ ]:
print (f"Opt.value = {model4.solver.value(model4.objective)}")

for _id_job, interval in enumerate(model4.jobs):
    _val = model4.solver.value(interval.start_expr())
    print(f"s[{_id_job}] = {_val}")

Opt.value = 19
s[0] = 0
s[1] = 0
s[2] = 0
s[3] = 3
s[4] = 5
s[5] = 4
s[6] = 6
s[7] = 12
s[8] = 14
s[9] = 6
s[10] = 9
s[11] = 11
s[12] = 14
s[13] = 19


In [ ]:
_filepath = data_dir / Path("pat104.rcp")
cond2 = Condition.from_file(_filepath)

In [ ]:
model5 = Model2Highs(cond2)
model5.solve()

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 941 rows; 9691 cols; 125568 nonzeros; 9639 integer variables (9639 binary)
Coefficient ranges:
  Matrix [1e+00, 2e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+02]
  RHS    [1e+00, 1e+01]


Presolving model


801 rows, 9551 cols, 124151 nonzeros  0s


795 rows, 7407 cols, 96558 nonzeros  1s


...

 B     129      16        32  62.64%   79              82                 3.66%     3214    119    540     29547    15.3s



Restarting search from the root node


Model after restart has 392 rows, 402 cols (371 bin., 0 int., 31 impl., 0 cont., 0 dom.fix.), and 4399 nonzeros



       133       0         0   0.00%   79              82                 3.66%       93      0      0     29610    15.6s
       133       0         0   0.00%   79              82                 3.66%       93     40      6     29826    15.7s


 L     133       0         0 100.00%   79              79                 0.00%     4510    164      6     30894    16.0s
       134       0         1 100.00%   79              79                 0.00%     4510    164      6     30973    16.0s

Solving report
  Status            Optimal
  Primal bound      79
  Dual bound        79
  Gap               0% (tolerance: 0.01%)
  P-D integral      1.91742902409
  Solution status   feasible
                    79 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            16.02 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 2
  Nodes             134
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     30973 (total)
                    1286 (strong br.)
                    13870 (separation)
                    11442 (heuristics)


In [ ]:
model6 = Model2CpSat(cond2)
model6.solve()


Starting CP-SAT solver v9.13.4784
Parameters: max_time_in_seconds: 180 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xe4d929acad86d0b6)
#Variables: 52 (#ints: 1 in objective) (51 primary variables)
  - 2 in [0,180]
  - 2 in [0,181]
  - 6 in [0,183]
  - 11 in [0,184]
  - 2 in [0,185]
  - 9 in [0,186]
  - 12 in [0,187]
  - 5 in [0,188]
  - 3 in [0,189]
#kCumulative: 3 (#intervals: 147)
#kInterval: 51
#kLinMax: 1 (#expressions: 51)
#kLinear2: 81

Starting presolve at 0.00s
  2.33e-05s  0.00e+00d  [DetectDominanceRelations] 
  1.75e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=11 #num_dual_strengthening=2 
  1.76e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  1.13e-05s  0.00e+00d  [DetectDuplicateColumns] 
  2.77e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 430 nodes and 650 arcs.
[Symmetry] Symmetry computation done. time: 4.19e-05 dtime: 6.216e-05
  2.53e-05s  0.00e+00d  [DetectDupli

#2       0.01s best:79    next:[78,78]    no_lp
#Done    0.01s no_lp
#Done    0.01s default_lp
#Done    0.01s quick_restart



Task timing                             n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                  'default_lp':         1 [  2.41ms,   2.41ms]   2.41ms   0.00ns   2.41ms         1 [ 58.11us,  58.11us]  58.11us   0.00ns  58.11us
            'feasibility_pump':         1 [  1.25ms,   1.25ms]   1.25ms   0.00ns   1.25ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                       'fixed':         1 [  2.70ms,   2.70ms]   2.70ms   0.00ns   2.70ms         1 [ 59.53us,  59.53us]  59.53us   0.00ns  59.53us
                          'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                          'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fs_random':         1 [826.01us, 826.01us] 826.01us   0.00ns 826.01us         0 [  0.00ns, 